In [1]:
import sqlite3
import pandas as pd
#from sqlalchemy import create_engine

In [2]:
%reload_ext sql

In [3]:
%sql sqlite:///databases/deliveroo.db

In [2]:
# Create connections to the databases
del_conn = sqlite3.connect(r'./databases/deliveroo.db')
take_conn = sqlite3.connect(r'./databases/takeaway.db')
uber_conn = sqlite3.connect(r'./databases/ubereats.db')

# Creat cursor objects
del_cur = del_conn.cursor()
take_cur = take_conn.cursor()
uber_cur = uber_conn.cursor()

- Which restaurants have the best price-rating ratio?

In [4]:
# 1. Deliveroo

# Chance '500+' to 500 in the restaurants table
del_cur.execute("""
    UPDATE restaurants
    SET rating_number = 500
    WHERE rating_number = '500+';
""")

# Convert rating_number to integer

del_cur.execute("""
    UPDATE restaurants
    SET rating_number = CAST(rating_number AS INTEGER);
""")

# Convert price to float
del_cur.execute("""
    UPDATE menu_items
    SET price = CAST(price AS REAL);
""")

# Delete rows with NULL in category
del_cur.execute("""
    DELETE FROM restaurants
    WHERE category IS NULL;
""")

# Get the top 5 restaurants with the best price-rating ratio
del_restaurants = del_cur.execute("""
    SELECT r.name AS rest_name, rating, ROUND(AVG(m.price), 2) AS avg_item_price
    FROM restaurants AS r
    INNER JOIN menu_items AS m
    ON r.id = m.restaurant_id
    GROUP BY rest_name
    ORDER BY rating DESC, avg_item_price ASC
    LIMIT 5;

""")
top5_deliveroo = del_restaurants.fetchall()
top5_deliveroo

[('Yokubō - Meal Kits', 5.0, 4.72),
 ('La Bagatelle de Lou', 5.0, 10.0),
 ('Neuhaus Woluwe Shopping', 5.0, 32.31),
 ('Moochie Aalst', 4.9, 1.72),
 ('The Kind Coconut', 4.9, 3.31)]

In [5]:
# 2. Takeaway

take_restaurants = take_cur.execute("""
    SELECT r.name AS rest_name, ratings AS rating, ROUND(AVG(m.price), 2) AS avg_item_price
    FROM restaurants AS r
    INNER JOIN menuItems AS m
    USING(primarySlug)
    GROUP BY rest_name
    ORDER BY ratings DESC, avg_item_price ASC
    LIMIT 5;
""")
top5_takeaway = take_restaurants.fetchall()
top5_takeaway

[('Le Corner', 5, 2.36),
 ('Pâtisserie Doce Tentação', 5, 2.63),
 ("Frituur op't Hoekske", 5, 2.86),
 ('QueTacos', 5, 2.86),
 ('JD-Corner', 5, 2.91)]

In [6]:
# 3. Uber Eats

# Correct price on the menu_items table
uber_cur.execute("""
    UPDATE menu_items
    SET price = price / 100;
""")

uber_restaurants = uber_cur.execute("""
    SELECT title AS rest_title, rating__rating_value AS rating, ROUND(AVG(price), 2) AS avg_item_price
    FROM restaurants AS r
    INNER JOIN menu_items AS m
    ON r.id = m.restaurant_id
    GROUP BY title
    ORDER BY rating__rating_value DESC, avg_item_price ASC    
    LIMIT 5;

""")
top5_uber = uber_restaurants.fetchall()
top5_uber

[("T'more enjoy", 5.0, 4.7),
 ('Boba Club', 5.0, 6.06),
 ('CrÃ¨merie Marie', 5.0, 7.68),
 ('JanYa Thai Wok', 5.0, 10.51),
 ('Carrefour Express - Merksem Lambrechtshoek', 4.9, 3.62)]

In [58]:
# What is the price distribution of menu items?

del_items = del_cur.execute("""
    SELECT *
    FROM menu_items;
                   
               """)

col_names = del_items.description
df = pd.DataFrame(del_items.fetchall())
df.columns = col_names
df['name'].value_counts()

(nan, nan, nan, nan, nan, nan)
Ketchup                           1298
Fanta                             1166
Sprite                             935
Mayonaise                          858
Andalouse                          707
                                  ... 
🥘 Thaïse currysoep 🌶️ (vegan🌱)       1
🥜 Poke Bowl Poulet Teriyaki          1
🥤 Coca Cola Zero 33cl                1
🥤 Coca cola 33cl                     1
🥩 Carpaccio                          1
Name: count, Length: 101791, dtype: int64

In [59]:
# How do delivery fees vary accross platforms and locations?

In [39]:
take_cursor.execute("""
    SELECT 1 AS name, 6 AS category, 7 AS address
                    FROM restaurants
""")

df_take_rest = pd.DataFrame(take_cursor.fetchall())
df_take_rest

,0,1,2
0,1,6,7
1,1,6,7
2,1,6,7
3,1,6,7
4,1,6,7
...,...,...,...
2711,1,6,7
2712,1,6,7
2713,1,6,7
2714,1,6,7
